# ** Linear Regression**

Regression은 하나의 종속변수 Y와 일련의 독립 변수 X 사이 관계의 형태를 탐색하고 결정하는 통계 측정방법입니다. 
$$y = a_0 + a_1 x_1 + a_2 x_2 + ... + a_n x_n$$

Linear regression은 모든 인스턴스에 대한 오차를 최소화하도록 계수를 최적화합니다.

오차는 아래의 수식으로 계산이 됩니다:

- Sum of residuals $\sum_{i = 1}^{n} (Y - \hat{Y})$
     - 양수 및 음수의 오류를 계산합니다.
- Sum of the absolute value of residuals $\sum_{i = 1}^{n} |Y - \hat{Y}|$
     - 오류차의 절대값의 합으로 계산합니다.
     - Outlier에 보다 robust 합니다.     
- Sum of square of residuals $\sum_{i = 1}^{n} (Y - \hat{Y})^2$
     - 오차값에 대해 보다 높은 패널티를 줍니다. 
     - Outlier에 위 수식보다 덜 robust 합니다.

### <u> Cost Function </u> 
Cost Function은 모델의 오류 값을 측정하는데 정의된 함수입니다.

- Cost function for MSE: $J_{\theta} = \frac{1}{n} \sum_{i = 1}^{n}(Y - \hat{Y})^2$

예측값을 높이기 위해, 우리는 cost function을 최소화해야 합니다. 이러한 목적으로 우리는 gradient descent algorithm을 사용하게 됩니다.

### <u> Gradient Descent </u>
Gradient Descent는 cost function 값이 최소화 되는 지점을 찾기 위해 파라미터들을 점진적으로 업데이트합니다.


### <u> Regression의 4가지 조건 </u>
    1. 선형성 : 설명 변수와 반응 변수 간의 관계 분포가 선형의 관계를 가진다.
    2. 독립성 : 설명 변수와 다른 설명 변수 간에 상관관계가 적다.
    3. 등분산성 : 잔차가 특정한 패턴을 보이지 않는다. (점점 커지거나 작아지는 패턴이 없다.)
    4. 정규성 : 잔차가 정규분포이다.    
    * 잔차 : 예측값 - 관측값 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from pathlib import Path
from IPython.display import display
from datetime import datetime
from pandas import DataFrame
from typing import List, NamedTuple, Tuple

# allow plots to appear directly in the notebook
%matplotlib 

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

ROOT_DIR = Path("/kaggle/input/bike-sharing-demand")
TRAIN_DATA_PATH = ROOT_DIR / "train.csv"
TEST_DATA_PATH = ROOT_DIR / "test.csv"

In [ ]:
def load(path: Path) -> DataFrame:
    return pd.read_csv(path, parse_dates=True, index_col="datetime")

def expanded_index_datetime_col(data: DataFrame) -> DataFrame:
    data = data.copy()
    data["hour"] = data.index.hour
    data["weekday"] = data.index.weekday
    data["month"] = data.index.month
    data["year"] = data.index.year
    return data


original_train: DataFrame = load(TRAIN_DATA_PATH)
#Not used in test data
original_train = original_train.drop(["casual", "registered"], axis=1, errors="ignore")
original_train = expanded_index_datetime_col(original_train)

display(original_train.head())

## 선형성
 - 설명 변수와 반응 변수 간의 관계 분포가 선형의 관계를 가진다.
 - 아래의 pairplot으로 선형성을 가지는 feature는 존재하지 않았다.

In [ ]:
sns.pairplot(original_train, x_vars=['weather', 'temp', 'atemp', 'humidity', 'windspeed'], y_vars='count'
            ,size=5, aspect=1)

## 독립성
 - 설명 변수와 다른 설명 변수 간에 상관관계(다중공선성)가 적다.
 - 피쳐간에 다중공선성이 존재하는 경우 모델이 각 피쳐와 대상 간의 관계를 독립적으로 추정하기가 어렵습니다. 
 - 상관관계가 높은 2개의 피쳐가 있는 경우, 하나의 피쳐를 삭제하거나 2개의 피쳐를 결합하여 새로운 피쳐를 만듦으로써 예측에 사용할 수 있습니다.
 - PairPlot과 heatmap(correlation matrix)를 통해 상관관계가 높은 특징들을 찾아 낼 수 있습니다.**** 

In [ ]:
sns.pairplot(original_train[['weather', 'temp', 'atemp', 'humidity', 'windspeed']])

In [ ]:
sns.heatmap(original_train[['weather', 'temp', 'atemp', 'humidity', 'windspeed']].corr(), annot=True)

위의 두 경우를 보면 temp와 atemp가 강한 상관관계를 보이는 것을 알 수 있습니다.

## 등분산성(homoscedasticity)
>  - 잔차가 특정한 패턴을 보이지 않습니다.
 - 잔차의 분포에 특정한 패턴이 있으면 데이터가 이분산성을 나타내고 이는 비선형성을 나타내게 됩니다.
 - 잔차는 독립변수의 값에 관계없이 동일한 분산을 가져야 합니다.
 - resid plot :선형 회귀의 잔차를 플로팅합니다. 이 함수는 x에서 y를 회귀하고 잔차의 산점도를 그립니다.

In [ ]:
# temp는 등분산성을 만족하지 않습니다.
sns.residplot(x=original_train['temp'], y=original_train['count'], lowess=True)

In [ ]:
# weather는 등분산성을 만족합니다.
sns.residplot(x=original_train['weather'], y=original_train['count'], lowess=True)

## 정규성
 - 잔차는 정규분포를 만족해야 합니다.
 - qq plot을 사용하여 데이터가 정규 분포에서 나온 것인지 유추 할 수 있습니다. 정규분포라면 플롯은 상당히 직선으로 나타납니다. 오차의 정규성 부재는 직선의 편차로 볼 수 있습니다.
 - 정규성 확인을 위해 간단한 Linear regression model을 만들어 확인해보도록 하겠습니다.

In [ ]:
x = original_train[['temp']]
y = original_train[["count"]]

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

model = LinearRegression()
result = model.fit(x_train, y_train)
y_pred = model.predict(x_test)

sns.distplot((y_test-y_pred),bins=100, color = 'gray')

## 모델의 성능
Linear Model의 성능 평가를 위해 우리는 $R^2$ 또는 $\text{Adjusted }R^2$ 값을 사용합니다.

$$R^2 = 1 - \frac{(Y - \hat{Y})^2}{(Y - \bar{Y})^2}$$

$R^2$ 은 모델이 얼마나 좋은지를 알려주는 값입니다.은$R^2$은 항상 0과 1사이의 값이며 1에 가까울 수록 높은 성능을 나타냅니다.

그러나 $R^2$ 의 문제점은 출력 변수와 관계없는 경우에도 더 많은 feature를 추가하면 하나의 feature일때와 동일한 값 혹은 증가한 값을 나타내게 됩니다. 그래서 우리는 $\text{Adjusted }R^2$ 를 같이 비교해야 합니다. $\text{Adjusted }R^2$ 는 기존 모델을 개선하지 않는 변수를 추가할 경우 불이익을 주게 됩니다.

$$\text{Adjusted }R^2 = 1 - \frac{(1 - R^2)(N - 1)}{N -P - 1}$$

여기서 $N$ 은 인스턴스의 수이며 $P$ 는 feature의 개수입니다.

따라서 여러 변수에 대해 선형 회귀를 작성하는 경우 항상 조정 된 R 제곱을 사용하여 모형의 우수성을 판단하는 것이 좋습니다. 입력 변수가 하나 뿐인 경우 R- 제곱과 조정 된 R 제곱은 정확히 같습니다.

일반적으로 모형에 유의하지 않은 변수를 더 많이 추가할수록 R- 제곱과 조정 된 R- 제곱의 간격이 증가합니다.

In [ ]:
def calculate_r2(model, x, y):
    return model.score(x, y)
    
def calculate_adjust_r2(model, x, y):
    r2 = model.score(x, y)
    N = len(x)
    P = len(x.columns)
    return 1-(1-r2)*(N-1)/(N-P-1)

def calculate_rmse(model, x, y):
    pred = model.predict(x)
    return np.sqrt(metrics.mean_squared_error(y, pred))

In [ ]:
print('R^2 : {:.3f}'.format(calculate_r2(model, x_test, y_test)))
print('Adjust R^2 : {:.3f}'.format(calculate_adjust_r2(model, x_test, y_test)))
print('RMSE : {:.2f}'.format(calculate_rmse(model, x_test, y_test)))

다음은 x의 feature를 여러 개로 설정하여 예측하는 모델을 테스트 합니다.

In [ ]:
def prepare(df: DataFrame) -> DataFrame:
    df = df.copy()
    df = replaced_with_onehot_cols(df, col_names=["season", "holiday", "workingday", "weather", "weekday", "month", "year"])
    df = df.drop(['atemp', 'windspeed'], axis=1)
    
    
    return df

def replaced_with_onehot_cols(data: DataFrame, col_names: List[str]) -> DataFrame:
    data = data.copy()
    
    for col_name in col_names:
        one_hot = pd.get_dummies(data[col_name], prefix=col_name)
        data = data.join(one_hot)
        
        # Original column is not needed anymore
        del data[col_name]
    return data

train_vals: DataFrame = prepare(original_train)

In [ ]:
x = train_vals.drop("count", axis=1)
y = train_vals[["count"]]

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

model = LinearRegression()

result = model.fit(x_train, y_train)

print('R^2 : {:.3f}'.format(calculate_r2(model, x_test, y_test)))
print('Adjust R^2 : {:.3f}'.format(calculate_adjust_r2(model, x_test, y_test)))
print('RMSE : {:.2f}'.format(calculate_rmse(model, x_test, y_test)))

# ** Lasso Regression**

Lasso는 Linear regression의 cost function에 L1-norm penalty를 주는 방식으로 cost 값을 계산하여 MSE와 penalty가 최소가 되도록 학습하는 목적을 가지고 있습니다.

### <u> Cost Function </u> 
- Cost function for MSE: $J_{\theta} = \frac{1}{n} \sum_{i = 1}^{n}(Y - \hat{Y})^2$
- Cost function for Lasso: $MSE + penalty = \frac{1}{n} \sum_{i = 1}^{n}(Y - \hat{Y})^2 + \alpha \sum_{j = 0}^{p}|w_j|$
- 여기서 m은 feature의 개수가 되고,  $\alpha$ 는 패널티의 니과를 조절해주는 파라미터 이다. $\alpha$ 가 작아지면 선형회귀와 같은 모형이 되고, $\alpha$ 가 커지면 패널티의 영향력이 커집니다. 

### 특성
- Lasso에서  $\alpha$ 는 하이퍼 파라미터로 사용자가 직접 지정을 해줘야 합니다. 
- Penalty를 통해 모델의 과적합을 막을 수 있습니다. 
- $w_j$가 0일 경우 해당 feature는 모델에 영향을 주지 않는 것을 앎으로써 (어떤 feature가 더 큰 영향을 미칠 수 있는지) 모델에 대한 해석력이 좋아집니다.


In [ ]:
from sklearn.linear_model import Lasso

model = Lasso(alpha=0.1, normalize=True)

model.fit(x_train, y_train)

print('R^2 : {:.3f}'.format(calculate_r2(model, x_test, y_test)))
print('Adjust R^2 : {:.3f}'.format(calculate_adjust_r2(model, x_test, y_test)))
print('RMSE : {:.2f}'.format(calculate_rmse(model, x_test, y_test)))

# ** Ridge Regression**

Ridge는 Lasso와 비슷하게 Linear regression의 cost function에 L2-norm penalty를 주는 방식으로 cost 값을 계산하여 MSE와 penalty가 최소가 되도록 학습하는 목적을 가지고 있습니다.

### <u> Cost Function </u> 
- Cost function for MSE: $J_{\theta} = \frac{1}{n} \sum_{i = 1}^{n}(Y - \hat{Y})^2$
- Cost function for Lasso: $MSE + penalty = \frac{1}{n} \sum_{i = 1}^{n}(Y - \hat{Y})^2 + \alpha \sum_{j = 0}^{p}(w_j)^2$
- 여기서 m은 feature의 개수가 되고,  $\alpha$ 는 패널티의 니과를 조절해주는 파라미터 이다. $\alpha$ 가 작아지면 선형회귀와 같은 모형이 되고, $\alpha$ 가 커지면 패널티의 영향력이 커집니다. 

### 특성
- Ridge에서 $\alpha$ 는 하이퍼 파라미터로 사용자가 직접 지정을 해줘야 합니다. 
- Penalty를 통해 모델의 과적합을 막을 수 있습니다. 
- $w_j$가 0에 가까울 경우 해당 feature는 모델에 영향을 주지 않는 것을 앎으로써 (어떤 feature가 더 큰 영향을 미칠 수 있는지) 모델에 대한 해석력이 좋아집니다.
- Lasso와의 다른점은 가중치 $w_j$가 0에 가까워질 뿐 0이 되지는 않습니다. 
- 많은 feature들 중 일부분만 중요하다면 Lasso가 더 높은 성능을 내고, 전체적으로 비슷하다면 Ridge가 더 높은 성능을 냅니다.

In [ ]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=0.1, normalize=True)

model.fit(x_train, y_train)

print('R^2 : {:.3f}'.format(calculate_r2(model, x_test, y_test)))
print('Adjust R^2 : {:.3f}'.format(calculate_adjust_r2(model, x_test, y_test)))
print('RMSE : {:.2f}'.format(calculate_rmse(model, x_test, y_test)))

# ** Elastic Net**

Llastic Net은 Ridge와 Lasso의 penalty를 모두 가지는 regression model입니다.

### <u> Cost Function </u> 
- Cost function for MSE: $J_{\theta} = \frac{1}{n} \sum_{i = 1}^{n}(Y - \hat{Y})^2$
- Cost function for ElasticNet: $MSE + penalty = \frac{1}{n} \sum_{i = 1}^{n}(Y - \hat{Y})^2 + \alpha \sum_{j = 0}^{p}(w_j)^2 + \alpha \sum_{j = 0}^{p}|w_j|$
- 여기서 m은 feature의 개수가 되고,  $\alpha$ 는 패널티의 니과를 조절해주는 파라미터 이다. $\alpha$ 가 작아지면 선형회귀와 같은 모형이 되고, $\alpha$ 가 커지면 패널티의 영향력이 커집니다. 


In [ ]:
from sklearn.linear_model import ElasticNet

model = ElasticNet(alpha=0.1, l1_ratio=0.1)

model.fit(x_train, y_train)

print('R^2 : {:.3f}'.format(calculate_r2(model, x_test, y_test)))
print('Adjust R^2 : {:.3f}'.format(calculate_adjust_r2(model, x_test, y_test)))
print('RMSE : {:.2f}'.format(calculate_rmse(model, x_test, y_test)))